# Code to gather data from $z > 5.5$ QSOs.

We want to obtain data from several surveys and catalogs in order to  
manipulate them and try to obtain meaningful correlations.

First, we import the packages to be used

To get the first line working, you need  
to run the following lines:

```bash
 conda install nodejs
 pip install ipympl
 pip install --upgrade jupyterlab
 jupyter labextension install @jupyter-widgets/jupyterlab-manager
 jupyter labextension install jupyter-matplotlib
 jupyter nbextension enable --py widgetsnbextension
 ```

In [1]:
%matplotlib inline
# Static plots
#%matplotlib ipympl
# Interactive plots
import numpy as np
# import matplotlib.cm as cm
# import matplotlib.pyplot as plt
# import matplotlib.colors as mcolors
# import matplotlib.patheffects as mpe
# import matplotlib.patheffects as path_effects
# from matplotlib.ticker import FormatStrFormatter
# from matplotlib.patches import Polygon
from astropy.io import fits
from astropy.table import Table
from astropy.table import Column
from astropy.table import MaskedColumn
from astropy.table import hstack
from astropy.table import vstack
from astropy.table import join
from astropy.wcs import WCS
from astropy import units as u
from astropy.cosmology import FlatLambdaCDM
from astropy.coordinates import SkyCoord
from astroquery.simbad import Simbad
from astroquery.ned import Ned
import getpass
import progressbar
import pandas as pd
import re

Define the spectral index $\alpha$ from different sources  
to be used in the luminosity calculations (K-correction)

In [2]:
alpha_first = 0.5  # From FIRST data (Bornancini+2010)
alpha_RG    = 1.0  # For radio galaxies (Verkhodanov & Khabibullina, 2010)
alpha_alex  = 0.8  # Star-forming galaxies (Alexander+2003)
alpha_smol  = 0.7  # Mean value from VLA-COSMOS 3GHz sample (Smolčić et al. 2017)
alpha_butl  = 0.75  # From Butler et al., 2018

Choose one of the spectral indexes

In [3]:
alpha_used  = alpha_butl

We define the cosmological properties to calculate luminosity distances and other quantities.

In [4]:
cosmo       = FlatLambdaCDM(H0=70, Om0=0.3)

In [5]:
def lum_from_flux(flux, redshift):  # Flux in mJy
    lum_distance = cosmo.luminosity_distance(redshift).to(u.m).value  # in m
    luminosity   = 4 * np.pi * lum_distance**2 * flux * 1e-3  * 1e-26 * (1 + redshift)**(alpha_used - 1)  # in W/Hz
    return luminosity

One function, to derive luminosity distances

In [6]:
def luminosity_distance(z_i, H0=70., WM=0.3, WV=0.7):
    z_i   = np.array([z_i], dtype='float64').flatten()
    c     = 299792.458 # velocity of light in km/sec
    h     = H0 / 100.
    WR    = 4.165E-5 / (h * h)   # includes 3 massless neutrino species, T0 = 2.72528
    WK    = 1 - WM - WR - WV
    azs   = 1.0 / (1 + z_i)
    DTT   = 0.0
    DCMR  = 0.0
    # do integral over a=1/(1+z) from az to 1 in n steps, midpoint rule
    n     = 1000  # number of points in integrals
    DL_Mpcs = np.zeros_like(z_i)
    for count, az in enumerate(azs):
        a     = az + (1 - az) * (np.arange(0, n) + 0.5) / n
        adot  = np.sqrt(WK + (WM / a) + (WR / (a * a)) + (WV * a * a))
        for i in range(n):
            # a    = az + (1 - az) * (i + 0.5) / n
            # adot = math.sqrt(WK + (WM / a) + (WR / (a * a)) + (WV * a * a))
            DTT  = DTT + 1. / adot[i]
            DCMR = DCMR + 1. / (a[i] * adot[i])
        DTT   = (1. - az) * DTT / n
        DCMR  = (1. - az) * DCMR / n
        # tangential comoving distance
        ratio = 1.00
        x     = np.sqrt(abs(WK)) * DCMR
        if x > 0.1:
            if WK > 0:
                ratio =  0.5 * (np.exp(x) - np.exp(-x)) / x 
            else:
                ratio = np.sin(x) / x
        else:
            y = x * x
        if WK < 0: y = -y
        ratio  = 1. + y / 6. + y * y / 120.
        DCMT   = ratio * DCMR
        DA     = az * DCMT
        DL     = DA / (az * az)
        DL_Mpc = (c / H0) * DL
        DL_Mpcs[count] = DL_Mpc
    return DL_Mpcs

---

### Reading data

Next step, reading our data.  
Most of the data files have been created using Topcat

In [7]:
machine  = getpass.getuser()
# cat_path = '/home/' + machine + '/Documentos/Data/'
cat_path = ''  # relative path to the same directory

~~Read data from FIRST+MILLIQUAS catalogs cross-matched.~~  
Read data from FIRST (from its version `14dec17`) crossmatched with  
SDSS Quasar Catalog (`SDSSDR14Q_v4_4`).

**FIRST + (SDSSQ)**   

Redshift values have been retrieved from the column `pipeline_redshift` in SDSS DR12  
The procedure to obtain these values is explained in **Bolton+2012**  

We also select sources with have explicitely data in both SDSS Quasar Catalog and  
in the FIRST survey (`first_match_flag = 1`).  

In order to get only the best redshift values, we select sources with  
`pipeline_redshift_flag = 0`.

Thus, we can use 11238 objects from the catalog.

In [8]:
# sdss_milli = Table.read(cat_path + 'tables_matches_milli_sdss_apr.fits')
# sdss_milli = Table.read(cat_path + 'milliquas_sdssquasar_jun2020.fits')

# L_14GHz_filter = np.array((sdss_milli['flux_20_cm'] > 0.0) * (sdss_milli['first_offset'] < 1.0)) ;  # sdss + milliquasar

# sdss_milli = Table.read(cat_path + 'first_14dec17_DR14Q_v4_4_zwarn_0.fits')  # SDSSQDR14+FIRST17Dec14
sdss_milli = Table.read(cat_path + 'first_14dec17_DR16Q_v4_zwarn_0.fits')  # SDSSQDR16+FIRST17Dec14

In [9]:
# sdss_milli.info

We load data from the VLA-COSMOS 3 GHz Large Project (**Smolčić et al. 2017**),  
which has been correlated with the sources from the VLA-COSMOS Large Project 1.4-GHz Source Catalog (**Schinnerer et al., 2007**).

Thus, we have only considered sources with observed (not estimated) $1.4$ GHz fluxes. That is, with valid  
values in the table column `flux_20cm`.

We can, also, separate sources which have an AGN X-ray counterpart
with the flag `cosm_xray_flag`

In [10]:
# cosmos_data         = Table.read(cat_path + 'vla_xmmcosmos_smolcic.fits')
cosmos_data         = Table.read(cat_path + 'vlacosxoid_vlacos3gh_xmmcosmos_ago2020.fits')

We want, also, to add $z > 6$ QSOs from the list in  
Table 3 in the review of **Inayoshi, Visbal, and Haiman, 2020**.  
Six of them have $z > 7$

Not all of them have $1.4$ GHz measurements. Others have  
measurements in different frequencies which can be translated  
into the desired frequency using, for instance, the relation  
from **Butler et al., 2018**:

$$S_{a} = S_{b} \times (\frac{\nu_{b}}{\nu_{a}})^{\alpha}$$  

We load the data from these sources. Fluxes from different frequencies than $1.4$ GHz are translated to the needed value.

In [11]:
# high_z_file       = 'high_z_qso_props_extended.tsv'  # 'high_z_qso_props.csv'

In [12]:
# high_z_ra         = np.char.replace(np.loadtxt(cat_path + high_z_file, usecols=[1],\
#                                                dtype='str', delimiter='\t'), ',', '.').astype(np.float)
# high_z_dec        = np.char.replace(np.loadtxt(cat_path + high_z_file, usecols=[2],\
#                                                dtype='str', delimiter='\t'), ',', '.').astype(np.float)
# high_z_zs         = np.char.replace(np.loadtxt(cat_path + high_z_file, usecols=[3],\
#                                                dtype='str', delimiter='\t'), ',', '.').astype(np.float)
# high_z_zs_e       = np.char.replace(np.loadtxt(cat_path + high_z_file, usecols=[4],\
#                                                dtype='str', delimiter='\t'), ',', '.').astype(np.float)
# high_z_f_14GHz    = np.char.replace(np.loadtxt(cat_path + high_z_file, usecols=[6],\
#                                                dtype='str', delimiter='\t'), ',', '.').astype(np.float)
# high_z_f_14GHz_e  = np.char.replace(np.loadtxt(cat_path + high_z_file, usecols=[7],\
#                                                dtype='str', delimiter='\t'), ',', '.').astype(np.float)
# high_z_f_3GHz     = np.char.replace(np.loadtxt(cat_path + high_z_file, usecols=[8],\
#                                                dtype='str', delimiter='\t'), ',', '.').astype(np.float)
# high_z_f_3GHz_e   = np.char.replace(np.loadtxt(cat_path + high_z_file, usecols=[9],\
#                                                dtype='str', delimiter='\t'), ',', '.').astype(np.float)
# high_z_f_15GHz    = np.char.replace(np.loadtxt(cat_path + high_z_file, usecols=[10],\
#                                                dtype='str', delimiter='\t'), ',', '.').astype(np.float)
# high_z_f_15GHz_e  = np.char.replace(np.loadtxt(cat_path + high_z_file, usecols=[11],\
#                                                dtype='str', delimiter='\t'), ',', '.').astype(np.float)
# high_z_f_250GHz   = np.char.replace(np.loadtxt(cat_path + high_z_file, usecols=[12],\
#                                                dtype='str', delimiter='\t'), ',', '.').astype(np.float)
# high_z_f_250GHz_e = np.char.replace(np.loadtxt(cat_path + high_z_file, usecols=[13],\
#                                                dtype='str', delimiter='\t'), ',', '.').astype(np.float)
# high_z_mass_1450  = np.char.replace(np.loadtxt(cat_path + high_z_file, usecols=[14],\
#                                                dtype='str', delimiter='\t'), ',', '.').astype(np.float)
# high_z_f_6cm      = np.zeros_like(high_z_zs)  # Keep empty
# high_z_f_6cm_e    = np.zeros_like(high_z_zs)  # Keep empty
# high_z_xmm_flux   = np.zeros_like(high_z_zs)
# high_z_xmm_flux_e = np.zeros_like(high_z_zs)
# high_z_names      = np.char.strip(np.loadtxt(cat_path + high_z_file, usecols=[0], dtype='str',\
#                                delimiter='\t'))
# high_z_names_alt  = np.char.strip(np.loadtxt(cat_path + high_z_file, usecols=[16], dtype='str',\
#                                delimiter='\t'))
# high_z_refs       = np.loadtxt(cat_path + high_z_file, usecols=[15], dtype='str',\
#                                delimiter='\t', encoding='utf-8')  # Article references for data
# # Limiting 1.4GHz flux from survey (mJy)
# high_z_flx_lim    = np.char.replace(np.loadtxt(cat_path + high_z_file, usecols=[3],\
#                                                dtype='str', delimiter='\t'), ',', '.').astype(np.float) * 1e-3
# high_z_lum_d      = luminosity_distance(high_z_zs) * 3.086e22  # in m
# high_z_up_lim     = np.array([val == '<' for val in np.loadtxt(cat_path + high_z_file,\
#                                                                usecols=[5], dtype='str', delimiter='\t')])
# # Flag to know origin of sources (0: SDSS, 1: COSMOS, 2: RADIO, 3: Stripe82, 4:Inayoshi)
# high_z_origin_flg = np.full_like(high_z_zs, fill_value=4, dtype=np.int)

In [13]:
# np.shape(high_z_zs)

In [14]:
high_z_sources    = Table.read(cat_path + 'milliquas_inayoshi_sdssqdr16_z55.fits')  # z >= 5.5 Inayoshi+Milliquas+SDSSQ

Cumulate values into one array except 250GHz data.  
Millimetre luminosities will be used separately since we cannot be completely  
sure that they represent, fully, non-thermal emission (from AGN) and not dust.

In [15]:
# high_z_14   = high_z_14GHz + high_z_3GHz + high_z_15GHz
# high_z_14_e = high_z_14GHz_e + high_z_3GHz_e + high_z_15GHz_e

To complement the dataset, we also load four $z > 5.5$ sources which  
come from the `radio` catalog in the `Heasarc` database.  
We queried the objects which have $1.4$ GHz observations that  
are within a $2.5$ arcsec of an object from the `SDSS QUASAR DR12`  
catalog. We discard the sources that are already included in the `FIRST`  
catalog, to avoid repetition.

To enlarge the size of the sample, we can also query  
the same sample, but extending the redshift range to all positive  
values. We can include, too, the exclusion of ***low quality*** redshift_values (or not).

In [16]:
# radio_sources        = Table.read(cat_path + 'radio_cat_sdss_z_55_spec.fits')       # high redshift
# radio_sources        = Table.read(cat_path + 'radio_cat_sdss_z_all_spec.fits')      # all redshift, high z quality
radio_sources        = Table.read(cat_path + 'radio_sdssquasar_jun2020.fits')      # all redshift, high z quality
# radio_sources        = Table.read(cat_path + 'radio_cat_sdss_z_all_all_spec.fits')  # all redshift, all z quality

We also include sources from the field `Stripe82` with `VLA` observations.  
The data has been obtained from **Hodge et al., 2011** (`VLASS821P4`; VLA SDSS Stripe 82 Survey 1.4-GHz Source Catalog).  
These sources have been crossmatched with the `SDSSQUASAR` catalog to obtain their redshift value.

In [17]:
# stripe82_sources     = Table.read(cat_path + 'vlass821p4_sdssquasar_jun2020.fits')
# stripe82_sources     = Table.read(cat_path + 'sdsss82cxo_sdsss82xmm_unique_jun2020.fits')
stripe82_sources     = Table.read(cat_path + 'J_ApJ_817_172_chandra_xmmao10_xmmao13_unique_vlass821p4_sdssquasar.fits')

---

### Calculate luminosities

In the future, we will calculate luminosities (in W/Hz) for different datasets  
using the expression

$$L_{1.4\mathrm{GHz}} = 4 \pi \mathrm{d}^{2}_{L} f_{1.4\mathrm{GHz}} (1 + z)^{\alpha - 1}$$

which comes from Alexander et al. 2003

We can also obtain that luminosity from the flux in $3$ GHz as

$$L_{1.4\mathrm{GHz}} = 4 \pi \mathrm{d}^{2}_{L} {(\frac{3}{1.4})}^{\alpha} f_{3\mathrm{GHz}} (1 + z)^{\alpha - 1}$$

This expression comes from Delhaize et al. 2017.

---

### Obtaining values from tables

In [18]:
stripe82_sources['RAdeg'][~(stripe82_sources['RAdeg'] > -999)] = 0.0
stripe82_sources['sdss_ra'][~(stripe82_sources['sdss_ra'] > -.999)] = 0.0
stripe82_sources['ra'] = stripe82_sources['RAdeg'].astype(np.float) + stripe82_sources['sdss_ra'].astype(np.float)

stripe82_sources['DEdeg'][~(stripe82_sources['DEdeg'] > -999.)] = 0.0
stripe82_sources['sdss_dec'][~(stripe82_sources['sdss_dec'] > -999.)] = 0.0
stripe82_sources['dec'] = stripe82_sources['DEdeg'].astype(np.float) + stripe82_sources['sdss_dec'].astype(np.float)

stripe82_sources['zsp'][~(stripe82_sources['zsp'] > 0)] = 0.0
stripe82_sources['sdss_z'][~(stripe82_sources['sdss_z'] > 0)] = 0.0
stripe82_sources['redshift'] = stripe82_sources['zsp'].astype(np.float) + stripe82_sources['sdss_z'].astype(np.float)
stripe82_sources['redshift'][(stripe82_sources['redshift'] <= 0)] = None
stripe82_sources['F1.4GHz'][~(stripe82_sources['F1.4GHz'] > 0)] = 0.0
stripe82_sources['strp82_20cm_flux'][~(stripe82_sources['strp82_20cm_flux'] > 0)] = 0.0
stripe82_sources['flux_20_cm'] = stripe82_sources['F1.4GHz'].astype(np.float) +\
stripe82_sources['strp82_20cm_flux'].astype(np.float)
stripe82_sources['e_F1.4GHz'][~(stripe82_sources['e_F1.4GHz'] > 0)] = 0.0
stripe82_sources['strp82_20cm_flux_err'][~(stripe82_sources['strp82_20cm_flux_err'] > 0)] = 0.0
stripe82_sources['flux_20_cm_error'] = stripe82_sources['e_F1.4GHz'].astype(np.float) +\
                                        stripe82_sources['strp82_20cm_flux_err'].astype(np.float)

# Do not use following filter. Include all sources
# filter_strp82    = np.array((stripe82_sources['flux_20_cm'] > 0) & (stripe82_sources['redshift'] > 0))
filter_strp82    = np.array(stripe82_sources['redshift'] > 0)
stripe82_sources = stripe82_sources[filter_strp82]

# L_14GHz_strp82   = lum_from_flux(stripe82_sources['strp82_20cm_flux'], stripe82_sources['sdss_z'])
# L_14GHz_strp82_e = np.abs(L_14GHz_strp82) * stripe82_sources['strp82_20cm_flux_err'] /\
# stripe82_sources['strp82_20cm_flux'];
# L_14GHz_strp82_e[~np.isfinite(L_14GHz_strp82_e)] = 0.0  # Repair (make zero) error values
L_14GHz_strp82   = lum_from_flux(stripe82_sources['flux_20_cm'], stripe82_sources['redshift'])
L_14GHz_strp82_e = np.abs(L_14GHz_strp82) * stripe82_sources['flux_20_cm_error'] / stripe82_sources['flux_20_cm'];
# L_14GHz_strp82_e[~np.isfinite(L_14GHz_strp82_e)] = 0.0  # Repair (make zero) error values

Now, we can use the points we are interested in. Our sample from **Inayoshi et al., 2020** and the  
sources from **SDSS+FIRST** with $z>5.5$.

Another option to display the data is, instead of showing redshift in the  
horizontal axis, have the mass of the observed objects.

**Inayoshi et al., 2020** use the rest-frame UV magnitude $\mathrm{M}_{1450}$  
to calculate the mass as:

$$M = 10^{[-(\mathrm{M}_{1450} + 3.459) / 2.5]} [\mathrm{M}_{\odot}]$$

which yields, on average, the published virial mass estimates for those available.

Create formal arrays from catalogs to merge them

Arrays from `SDSS+FIRST`

In [19]:
#limit_z_sdss_up       = 5.5
limit_z_sdss          = 0.0
filter_sdss_z         = np.array(sdss_milli['Z'] > limit_z_sdss)
xmm_freq              = 1.47e18  # Hz. Frequency for 0.2-12 keV observations

In [20]:
upper_sdss            = sdss_milli[np.array(sdss_milli['Z'] > limit_z_sdss)]
upper_sdss_ra         = upper_sdss['RA_1']
upper_sdss_dec        = upper_sdss['DEC_1']
# upper_sdss_z          = upper_sdss['Z_PIPE']
# upper_sdss_z_e        = upper_sdss['Z_PIPE_ERR']
upper_sdss_z          = upper_sdss['Z']
upper_sdss_z_e        = np.zeros_like(upper_sdss_z)
upper_sdss_f_20cm     = upper_sdss['FPEAK']  # mJy
upper_sdss_f_20cm_e   = upper_sdss['RMS']  # mJy
upper_sdss_u_lim      = np.zeros_like(upper_sdss_z, dtype=np.bool)
upper_sdss_f_6cm      = np.zeros_like(upper_sdss_z)
upper_sdss_f_6cm_e    = np.zeros_like(upper_sdss_z)
upper_sdss_f_3GHz     = np.zeros_like(upper_sdss_z)
upper_sdss_f_3GHz_e   = np.zeros_like(upper_sdss_z)
upper_sdss_f_15GHz    = np.zeros_like(upper_sdss_z)  # 1.5 GHz
upper_sdss_f_15GHz_e  = np.zeros_like(upper_sdss_z)
upper_sdss_f_250GHz   = np.zeros_like(upper_sdss_z)
upper_sdss_f_250GHz_e = np.zeros_like(upper_sdss_z)
upper_sdss_xmm_flux   = upper_sdss['XMM_TOTAL_FLUX'] / xmm_freq * 1e23 * 1e3     # erg cm-2 s-1. Convert to mJy
upper_sdss_xmm_flux_e = upper_sdss['XMM_TOTAL_FLUX_ERR'] / xmm_freq * 1e23 * 1e3 # erg cm-2 s-1. Convert to mJy
upper_sdss_mass_1450  = np.zeros_like(upper_sdss_z)
upper_sdss_flx_lim    = np.array([0.150]*np.shape(upper_sdss_z)[0])  # Limiting 1.4GHz flux from survey (mJy)
# Flag to know origin of sources (0: SDSS, 1: COSMOS, 2: RADIO, 3: Stripe82, 4:Inayoshi)
upper_sdss_origin_flg = np.zeros_like(upper_sdss_z, dtype=np.int)
upper_sdss_refs       = np.array(['Lyke+2020,Paris+2018,Helfand+2015'.encode('utf-8')] * np.shape(upper_sdss_z)[0])

In [21]:
np.shape(upper_sdss_z)

(13396,)

Arrays from the `COSMOS` Field

In [22]:
upper_cosmos            = cosmos_data[np.array(cosmos_data['cosm_z'] > limit_z_sdss)]
upper_cosmos_ra         = upper_cosmos['cosm_ra']
upper_cosmos_dec        = upper_cosmos['cosm_dec']
upper_cosmos_z          = upper_cosmos['cosm_z']
upper_cosmos_z_e        = np.zeros_like(upper_cosmos_z)
upper_cosmos_f_20cm     = upper_cosmos['flux_20cm']      # mJy
upper_cosmos_f_20cm_e   = upper_cosmos['flux_20cm_err']  # mJy
upper_cosmos_u_lim      = np.zeros_like(upper_cosmos_z, dtype=np.bool)
upper_cosmos_f_3GHz     = upper_cosmos['cosm_f_3ghz']
upper_cosmos_f_3GHz_e   = upper_cosmos['cosm_f_3ghz_err']
upper_cosmos_f_6cm      = np.zeros_like(upper_cosmos_z)
upper_cosmos_f_6cm_e    = np.zeros_like(upper_cosmos_z)
upper_cosmos_f_15GHz    = np.zeros_like(upper_cosmos_z)  # 1.5 GHz
upper_cosmos_f_15GHz_e  = np.zeros_like(upper_cosmos_z)
upper_cosmos_f_250GHz   = np.zeros_like(upper_cosmos_z)
upper_cosmos_f_250GHz_e = np.zeros_like(upper_cosmos_z)
upper_cosmos_xmm_flux   = upper_cosmos['S.5-10'] * 1e-17 / xmm_freq * 1e23 * 1e3    # 1e-17 W m-2. Convert to mJy
upper_cosmos_xmm_flux_e = upper_cosmos['e_S.5-10'] * 1e-17 / xmm_freq * 1e23 * 1e3  # 1e-17 W m-2. Convert to mJy
upper_cosmos_mass_1450  = np.zeros_like(upper_cosmos_z)
upper_cosmos_flx_lim    = np.array([0.045] * np.shape(upper_cosmos_z)[0])  # Limiting 1.4GHz flux from survey (mJy)
# Flag to know origin of sources (0: SDSS, 1: COSMOS, 2: Stripe82, 3:Inayoshi)
upper_cosmos_origin_flg = np.ones_like(upper_cosmos_z, dtype=np.int)
upper_cosmos_refs       = np.array(['Smolcic+2017,Cappelluti+2009'.encode('utf-8')] * np.shape(upper_cosmos_z)[0])

In [23]:
np.shape(upper_cosmos_z)

(2433,)

Arrays from `vlass821p4` catalog.

In [24]:
strp82_ra                     = stripe82_sources['RA']                # in degree
strp82_dec                    = stripe82_sources['DEC']               # in degree
strp82_u_lim                  = np.zeros_like(strp82_ra, dtype=np.bool)
strp82_z                      = stripe82_sources['redshift']
strp82_z.mask                 = np.array(strp82_z <= 0)               # Fix format for column
strp82_z.fill_value           = np.nan                                # Fix format for column
# strp82_z_e           = stripe82_sources['sdss_z_err']
strp82_z_e                    = np.zeros_like(strp82_z)               # Spectroscopic redshift
strp82_20cm_flux              = stripe82_sources['flux_20_cm']        # in mJy
strp82_20cm_flux.mask         = np.array(strp82_20cm_flux <= 0)
strp82_20cm_flux.fill_value   = np.nan
strp82_20cm_flux_e            = stripe82_sources['flux_20_cm_error']  # in mJy
strp82_20cm_flux_e.mask       = np.array(strp82_20cm_flux <= 0)
strp82_20cm_flux_e.fill_value = np.nan
strp82_f_3GHz                 = np.zeros_like(strp82_z)
strp82_f_3GHz_e               = np.zeros_like(strp82_z)
strp82_f_6cm                  = np.zeros_like(strp82_z)
strp82_f_6cm_e                = np.zeros_like(strp82_z)
strp82_f_15GHz                = np.zeros_like(strp82_z)  # 1.5 GHz
strp82_f_15GHz_e              = np.zeros_like(strp82_z)
strp82_f_250GHz               = np.zeros_like(strp82_z)
strp82_f_250GHz_e             = np.zeros_like(strp82_z)

stripe82_sources['FFull'][~(stripe82_sources['FFull'] > 0)] = 0.0
stripe82_sources['xmm_flux'][~(stripe82_sources['xmm_flux'] > 0)] = 0.0
stripe82_sources['fb_flux'] = stripe82_sources['FFull'] * 1e3 * 1e-17 + stripe82_sources['xmm_flux']  # 1e-17 W/m2
stripe82_sources['e_FFull'][~(stripe82_sources['e_FFull'] > 0)] = 0.0
stripe82_sources['xmm_flux_err'][~(stripe82_sources['xmm_flux_err'] > 0)] = 0.0
stripe82_sources['fb_flux_e'] = stripe82_sources['e_FFull'] * 1e3 * 1e-17 + stripe82_sources['xmm_flux_err']

strp82_xmm_flux               = stripe82_sources['fb_flux'] / xmm_freq * 1e23 * 1e3  # erg cm-2 s-1. Convert to mJy
strp82_xmm_flux.mask          = np.array(strp82_xmm_flux <= 0)
strp82_xmm_flux.fill_value    = np.nan
strp82_xmm_flux_e             = stripe82_sources['fb_flux_e'] / xmm_freq * 1e23 * 1e3  # erg cm-2 s-1. Convert to mJy
strp82_xmm_flux_e.mask        = np.array(strp82_xmm_flux_e <= 0)
strp82_xmm_flux_e.fill_value  = np.nan
# strp82_spec_index    = stripe82_sources['spec_index']
strp82_spec_index   = np.zeros_like(strp82_z)
strp82_mass_1450    = np.zeros_like(strp82_z)
strp82_flx_lim      = np.array([0.052] * np.shape(strp82_z)[0])  # Limiting 1.4GHz flux from survey (mJy)
# Flag to know origin of sources (0: SDSS, 1: COSMOS, 2: Stripe82, 3:Inayoshi)
strp82_origin_flg   = np.full_like(strp82_z, fill_value=2, dtype=np.int)
strp82_refs         = np.array(['Lyke+2020,Hodge+2011,LaMassa+2016,Paris+2017'.encode('utf-8')] * np.shape(strp82_z)[0])

In [25]:
np.shape(strp82_z)

(1838,)

Arrays from the Extended **Inayoshi et al., 2020** catalog ($z > 5.5$).

In [26]:
high_z_ra         = high_z_sources['RA']
high_z_dec        = high_z_sources['DEC']
high_z_zs         = high_z_sources['Z']
high_z_zs_e       = high_z_sources['Z_e']
high_z_f_14GHz    = high_z_sources['F20cm'] * 1e-3
high_z_f_14GHz_e  = high_z_sources['F20cmE'] * 1e-3
filter_yes14      = np.array(high_z_f_14GHz > 0.)
filter_good_f     = np.array(high_z_sources['FIRST_FLUX'] > 0.)
# Add FIRST measurements where available
high_z_f_14GHz[~filter_yes14 & filter_good_f]   = high_z_sources['FIRST_FLUX'][~filter_yes14 & filter_good_f]
high_z_f_14GHz_e[~filter_yes14 & filter_good_f] = (high_z_sources['FIRST_FLUX'][~filter_yes14 & filter_good_f]\
                                                   - 0.25) / high_z_sources['FIRST_SNR'][~filter_yes14 & filter_good_f]
high_z_f_3GHz     = high_z_sources['F3GHz'] * 1e-3
high_z_f_3GHz_e   = high_z_sources['F3GHzE'] * 1e-3
high_z_f_15GHz    = high_z_sources['F1.5GHz'] * 1e-3
high_z_f_15GHz_e  = high_z_sources['F1.5GHzE'] * 1e-3
high_z_f_250GHz   = high_z_sources['F250GHz'] * 1e-3  # mJy
high_z_f_250GHz_e = high_z_sources['F250GHzE'] * 1e-3  # mJy
high_z_mass_1450  = high_z_sources['BH_MASS']
high_z_f_6cm      = np.zeros_like(high_z_zs)  # Keep empty
high_z_f_6cm_e    = np.zeros_like(high_z_zs)  # Keep empty
high_z_xmm_flux   = high_z_sources['XMM_TOTAL_FLUX'] / xmm_freq * 1e23 * 1e3  # erg cm-2 s-1. Convert to mJy
high_z_xmm_flux_e = high_z_sources['XMM_TOTAL_FLUX_ERR'] / xmm_freq * 1e23 * 1e3  # erg cm-2 s-1. Convert to mJy
high_z_names      = high_z_sources['Name_2']
high_z_names_alt  = high_z_sources['Alt_Name']
filter_empty_ref  = np.array(np.char.strip(high_z_sources['REF'].astype(np.str)) == '+')
high_z_refs       = np.char.strip(high_z_sources['REF'].astype(np.str))  # Article references for data
high_z_refs[filter_empty_ref] = 'Lyke+2020,Paris+2018,Helfand+2015'.encode('utf-8')  # They come from SDSS+FIRST
high_z_flx_lim    = high_z_sources['OBS_LIM'] * 1e-3 # Limiting 1.4GHz flux from survey (mJy)
high_z_lum_d      = luminosity_distance(high_z_zs) * 3.086e22  # in m
high_z_up_lim     = high_z_sources['up20cm']  # Boolean values (True if it is an upper limit)
# Flag to know origin of sources (0: SDSS, 1: COSMOS, 2: Stripe82, 3:Inayoshi)
high_z_origin_flg = np.full_like(high_z_zs, fill_value=3, dtype=np.int)

In [27]:
np.shape(high_z_zs)

(2312,)

Merging `SDSS+FIRST` and the extended catalog from **Inayoshi et al., 2020**

In [28]:
medium_sample_ra         = np.append(upper_sdss_ra,         high_z_ra)          # deg
medium_sample_dec        = np.append(upper_sdss_dec,        high_z_dec)         # deg
medium_sample_u_lim      = np.append(upper_sdss_u_lim,      high_z_up_lim)
medium_sample_f20cm      = np.append(upper_sdss_f_20cm,     high_z_f_14GHz)     # mJy
medium_sample_f20cm_e    = np.append(upper_sdss_f_20cm_e,   high_z_f_14GHz_e)   # mJy

medium_sample_f_6cm      = np.append(upper_sdss_f_6cm,      high_z_f_6cm)       # mJy
medium_sample_f_6cm_e    = np.append(upper_sdss_f_6cm_e,    high_z_f_6cm_e)     # mJy
medium_sample_f_3GHz     = np.append(upper_sdss_f_3GHz,     high_z_f_3GHz)      # mJy
medium_sample_f_3GHz_e   = np.append(upper_sdss_f_3GHz_e,   high_z_f_3GHz_e)    # mJy
medium_sample_f_15GHz    = np.append(upper_sdss_f_15GHz,    high_z_f_15GHz)     # mJy
medium_sample_f_15GHz_e  = np.append(upper_sdss_f_15GHz_e,  high_z_f_15GHz_e)   # mJy

medium_sample_f250GHz    = np.append(upper_sdss_f_250GHz,   high_z_f_250GHz)    # mJy
medium_sample_f250GHz_e  = np.append(upper_sdss_f_250GHz_e, high_z_f_250GHz_e)  # mJy
medium_sample_z          = np.append(upper_sdss_z,          high_z_zs)
medium_sample_z_e        = np.append(upper_sdss_z_e,        high_z_zs_e)
medium_sample_xmm_flux   = np.append(upper_sdss_xmm_flux,   high_z_xmm_flux)    # erg cm-2 s-1
medium_sample_xmm_flux_e = np.append(upper_sdss_xmm_flux_e, high_z_xmm_flux_e)  # erg cm-2 s-1
medium_sample_mass_1450  = np.append(upper_sdss_mass_1450,  high_z_mass_1450)   # M_sun
medium_sample_flx_lim    = np.append(upper_sdss_flx_lim,    high_z_flx_lim)     # mJy
medium_sample_origin_flg = np.append(upper_sdss_origin_flg, high_z_origin_flg)
medium_sample_refs       = np.append(upper_sdss_refs,       high_z_refs)

In [29]:
np.shape(medium_sample_z)

(15708,)

Merging `SDSS+FIRST`+**Inayoshi et al., 2020** and `VLASS821P4` data

In [30]:
medium82_sample_ra         = np.append(medium_sample_ra,         strp82_ra)                     # deg
medium82_sample_dec        = np.append(medium_sample_dec,        strp82_dec)                    # deg
medium82_sample_u_lim      = np.append(medium_sample_u_lim,      strp82_u_lim)
medium82_sample_f20cm      = np.append(medium_sample_f20cm,      strp82_20cm_flux.filled(0))    # mJy
medium82_sample_f20cm_e    = np.append(medium_sample_f20cm_e,    strp82_20cm_flux_e.filled(0))  # mJy

medium82_sample_f_6cm      = np.append(medium_sample_f_6cm,      strp82_f_6cm.filled(0))        # mJy
medium82_sample_f_6cm_e    = np.append(medium_sample_f_6cm_e,    strp82_f_6cm_e.filled(0))      # mJy
medium82_sample_f_3GHz     = np.append(medium_sample_f_3GHz,     strp82_f_3GHz.filled(0))       # mJy
medium82_sample_f_3GHz_e   = np.append(medium_sample_f_3GHz_e,   strp82_f_3GHz_e.filled(0))     # mJy
medium82_sample_f_15GHz    = np.append(medium_sample_f_15GHz,    strp82_f_15GHz.filled(0))      # mJy
medium82_sample_f_15GHz_e  = np.append(medium_sample_f_15GHz_e,  strp82_f_15GHz_e.filled(0))    # mJy

medium82_sample_f250GHz    = np.append(medium_sample_f250GHz,    strp82_f_250GHz.filled(0))     # mJy
medium82_sample_f250GHz_e  = np.append(medium_sample_f250GHz_e,  strp82_f_250GHz_e.filled(0))   # mJy
medium82_sample_z          = np.append(medium_sample_z,          strp82_z.filled(0))
medium82_sample_z_e        = np.append(medium_sample_z_e,        strp82_z_e.filled(0))
medium82_sample_xmm_flux   = np.append(medium_sample_xmm_flux,   strp82_xmm_flux.filled(0))     # in erg cm-2 s-1
medium82_sample_xmm_flux_e = np.append(medium_sample_xmm_flux_e, strp82_xmm_flux_e.filled(0))   # in erg cm-2 s-1
medium82_sample_mass_1450  = np.append(medium_sample_mass_1450,  strp82_mass_1450.filled(0))    # M_sun
medium82_sample_flx_lim    = np.append(medium_sample_flx_lim,    strp82_flx_lim)                # mJy
medium82_sample_origin_flg = np.append(medium_sample_origin_flg, strp82_origin_flg)
medium82_sample_refs       = np.append(medium_sample_refs,       strp82_refs)

In [31]:
np.shape(medium82_sample_z)

(17546,)

Merging `SDSS+FIRST`+**Inayoshi et al., 2020**+`VLASS821P4` and `COSMOS` data

In [32]:
large_sample_ra         = np.append(medium82_sample_ra,         upper_cosmos_ra)          # deg
large_sample_dec        = np.append(medium82_sample_dec,        upper_cosmos_dec)         # deg
large_sample_u_lim      = np.append(medium82_sample_u_lim,      upper_cosmos_u_lim)
large_sample_f20cm      = np.append(medium82_sample_f20cm,      upper_cosmos_f_20cm)      # mJy
large_sample_f20cm_e    = np.append(medium82_sample_f20cm_e,    upper_cosmos_f_20cm_e)    # mJy

large_sample_f_6cm      = np.append(medium82_sample_f_6cm,      upper_cosmos_f_6cm)       # mJy
large_sample_f_6cm_e    = np.append(medium82_sample_f_6cm_e,    upper_cosmos_f_6cm_e)     # mJy
large_sample_f_3GHz     = np.append(medium82_sample_f_3GHz,     upper_cosmos_f_3GHz)      # mJy
large_sample_f_3GHz_e   = np.append(medium82_sample_f_3GHz_e,   upper_cosmos_f_3GHz_e)    # mJy
large_sample_f_15GHz    = np.append(medium82_sample_f_15GHz,    upper_cosmos_f_15GHz)     # mJy
large_sample_f_15GHz_e  = np.append(medium82_sample_f_15GHz_e,  upper_cosmos_f_15GHz_e)   # mJy

large_sample_f250GHz    = np.append(medium82_sample_f250GHz,    upper_cosmos_f_250GHz)    # mJy
large_sample_f250GHz_e  = np.append(medium82_sample_f250GHz_e,  upper_cosmos_f_250GHz_e)  # mJy
large_sample_z          = np.append(medium82_sample_z,          upper_cosmos_z)
large_sample_z_e        = np.append(medium82_sample_z_e,        upper_cosmos_z_e)
large_sample_xmm_flux   = np.append(medium82_sample_xmm_flux,   upper_cosmos_xmm_flux)    # in erg cm-2 s-1
large_sample_xmm_flux_e = np.append(medium82_sample_xmm_flux_e, upper_cosmos_xmm_flux_e)  # in erg cm-2 s-1
large_sample_mass_1450  = np.append(medium82_sample_mass_1450,  upper_cosmos_mass_1450)   # M_sun
large_sample_flx_lim    = np.append(medium82_sample_flx_lim,    upper_cosmos_flx_lim)     # mJy
large_sample_origin_flg = np.append(medium82_sample_origin_flg, upper_cosmos_origin_flg)
large_sample_refs       = np.append(medium82_sample_refs,       upper_cosmos_refs)

In [33]:
np.shape(large_sample_z)

(19979,)

At this point, we also want to obtain more properties from the selected  
sources (**Inayoshi et al., 2020** + **SDSS+FIRST**). We will use `astroquery` to  
obtain information from `simbad`.

First, we obtain the names and coordinates of our sources to query them.

In [34]:
small_sample_names  = np.append(upper_sdss['SDSS_NAME'], high_z_names)
medium_sample_names = np.append(small_sample_names, stripe82_sources['Coord_name'])
large_sample_names  = np.append(medium_sample_names, upper_cosmos['cosm_name'])
large_sample_coords = SkyCoord(ra=large_sample_ra, dec=large_sample_dec, unit=(u.deg, u.deg))

In [35]:
np.shape(large_sample_names)

(19979,)

Create `MaskedColumns` (from `Astropy`) using the data just collected.

In [36]:
if True:
    column_cat_index    = MaskedColumn(np.arange(np.shape(large_sample_z)[0]), name='INDEX', dtype='int',\
                                       description='Idx_n')
    column_cat_name     = MaskedColumn(large_sample_names.data, name='CAT_NAME', dtype='str', description='Cat_name',\
                                       mask=np.array(large_sample_names.data == ''))
    column_cat_ra       = MaskedColumn(large_sample_coords.ra.degree, name='RA', description='RA',\
                                       mask=~np.isfinite(large_sample_coords.ra.degree), fill_value=np.nan)
    column_cat_dec      = MaskedColumn(large_sample_coords.dec.degree, name='DEC', description='DEC',\
                                       mask=~np.isfinite(large_sample_coords.dec.degree), fill_value=np.nan)
    column_z_own        = MaskedColumn(large_sample_z.data, name='Z_OWN', unit='', description='z', fill_value=np.nan,\
                                       mask=np.array(large_sample_z.data == 0))
    column_z_own_err    = MaskedColumn(large_sample_z_e.data, name='Z_OWN_ERR', unit='', description='z_err',\
                                       fill_value=np.nan, mask=np.array(large_sample_z_e.data == 0))
    column_L_14GHz_up   = MaskedColumn(large_sample_u_lim, name='L_20CM_UP_LIM', dtype='bool', description='L20cmUp')
    column_f_20cm       = MaskedColumn(large_sample_f20cm.data, name='F_20CM', unit='mJy', description='20cmF',\
                                       fill_value=np.nan, mask=np.array(large_sample_f20cm.data == 0))
    column_f_20cm_err   = MaskedColumn(large_sample_f20cm_e.data, name='F_20CM_ERR', unit='mJy', description='20cmFe',\
                                       fill_value=np.nan, mask=np.array(large_sample_f20cm_e.data == 0))
    column_f_6cm        = MaskedColumn(large_sample_f_6cm.data, name='F_6CM', unit='mJy', description='6cmF',\
                                       fill_value=np.nan, mask=np.array(large_sample_f_6cm.data == 0))
    column_f_6cm_err    = MaskedColumn(large_sample_f_6cm_e.data, name='F_6CM_ERR', unit='mJy', description='6cmFe',\
                                       fill_value=np.nan, mask=np.array(large_sample_f_6cm_e.data == 0))
    column_f_3GHz       = MaskedColumn(large_sample_f_3GHz.data, name='F_3GHZ', unit='mJy', description='3GHzF',\
                                       fill_value=np.nan, mask=np.array(large_sample_f_3GHz.data == 0))
    column_f_3GHz_err   = MaskedColumn(large_sample_f_3GHz_e.data, name='F_3GHZ_ERR', unit='mJy', description='3GHzFe',\
                                       fill_value=np.nan, mask=np.array(large_sample_f_3GHz_e.data == 0))
    column_f_15GHz      = MaskedColumn(large_sample_f_15GHz.data, name='F_1.5GHZ', unit='mJy', description='1.5GHzF',\
                                       fill_value=np.nan, mask=np.array(large_sample_f_15GHz.data == 0))
    column_f_15GHz_err  = MaskedColumn(large_sample_f_15GHz_e.data, name='F_1.5GHZ_ERR', unit='mJy', description='1.5GHzFe',\
                                       fill_value=np.nan, mask=np.array(large_sample_f_15GHz_e.data == 0))
    column_f_xmm        = MaskedColumn(large_sample_xmm_flux.data, name='F_XMM', unit='mJy', description='F_xmm',\
                                       fill_value=np.nan,\
                                       mask=np.array(large_sample_xmm_flux.data == 0 & np.isnan(large_sample_xmm_flux.data)))
    column_f_xmm_err    = MaskedColumn(large_sample_xmm_flux_e.data, name='F_XMM_ERR', unit='mJy', description='F_xmme',\
                                       fill_value=np.nan,\
                                       mask=np.array(large_sample_xmm_flux_e.data == 0 & np.isnan(large_sample_xmm_flux_e.data)))
    column_f_250GHz     = MaskedColumn(large_sample_f250GHz.data, name='F_250GHZ', unit='mJy', description='250GHzF',\
                                       fill_value=np.nan, mask=np.array(large_sample_f250GHz.data == 0))
    column_f_250GHz_err = MaskedColumn(large_sample_f250GHz_e.data, name='F_250GHZ_ERR', unit='mJy', description='250GHzFe',\
                                       fill_value=np.nan, mask=np.array(large_sample_f250GHz_e.data == 0))
    column_mass_1450    = MaskedColumn(large_sample_mass_1450.data, name='MASS_1450', unit='Msun', description='UVmass',\
                                       fill_value=np.nan, mask=np.array(large_sample_mass_1450.data == 0))
    column_flx_lim      = MaskedColumn(large_sample_flx_lim.data, name='FLX_LIM', unit='mJy', description='flx_lim',\
                                       fill_value=np.nan, mask=np.array(large_sample_flx_lim.data == 0))
    column_origin       = MaskedColumn(large_sample_origin_flg.data, name='ORIGIN', unit='', description='origin',\
                                       dtype='int', mask=~np.isfinite(large_sample_origin_flg.data))
    column_refs         = MaskedColumn(large_sample_refs.data, name='REFS', unit='', description='Refs',\
                                       dtype=np.str, fill_value=np.nan)

Then, we can query the database to obtain the desired data.  In this point,  
we also add more columns to be queried.

In [37]:
query_simbad_flag               = False
load_simbad_flag                = False
create_simbad_inayoshi_flag     = False
read_simbad_inayoshi_flag       = False
query_ned_names_flag            = False
query_ned_photometry_flag       = False
create_phot_bands_list_flag     = False
load_phot_bands_list_flag       = False
order_ned_photometry_flag       = False
create_simbad_inayoshi_ned_flag = False
read_simbad_inayoshi_ned_flag   = False

In [38]:
if query_simbad_flag:
    customSimbad   = Simbad()
    initial_fields = customSimbad.get_votable_fields()

    if 'coordinates' in initial_fields:
        customSimbad.remove_votable_fields('coordinates')
        customSimbad.add_votable_fields('ra(d)', 'dec(d)')
    if 'z_value' not in initial_fields:
        customSimbad.add_votable_fields('z_value')
    for band in ['B','V','R','I','J','K']:
        if f'fluxdata({band})' not in initial_fields:
            customSimbad.add_votable_fields(f'flux({band})', f'flux_error({band})')

All sources but those from `COSMOS` catalog have meaningful (for `simbad`) names.  
Thus, separate queries will be executed. And, to standardize results, queries  
will only be based on coordinates (not names).

In [39]:
if query_simbad_flag:
    limit_set            = int(np.floor(np.shape(medium82_sample_z)[0]/3))

In [40]:
if query_simbad_flag:
    result_table_simbad  = Table()

In [41]:
# result_table = customSimbad.query_objects(large_sample_names)
# customSimbad.TIMEOUT = 240

# result_table_job_a   = customSimbad.query_objects(large_sample_names[:limit_set])

# result_table_job_b   = customSimbad.query_objects(large_sample_names[limit_set:(limit_set*2)])

# result_table_job_c   = customSimbad.query_objects(large_sample_names[(limit_set*2):(limit_set*3)])

In [42]:
if query_simbad_flag:
    query_error = 0
    final_error = 0

Query `SDSS+FIRST` (`milliquasar`) + **Inayoshi et al., 2020** + `VLASS821P4` sources.

In [43]:
if query_simbad_flag:
    with progressbar.ProgressBar(min_value=0, max_value=(limit_set - 1)) as bar:
        for index, coord in enumerate(large_sample_coords[:limit_set]):
            temp_table      = Table()
            try:
                temp_table      = customSimbad.query_region(coord, radius=2.0*u.arcsec)
                if len(temp_table) == 0:
                    temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype,\
                                                                            result_table_simbad['DEC_d'].info.dtype))
                    temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
            except:
                temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype,\
                                                                        result_table_simbad['DEC_d'].info.dtype))
                temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
                #print(f'Error with element {(index)} of the sample')
                query_error += 1
            try:
                if temp_table['RA_d'].unit != 'deg':
                    replacement_col     = MaskedColumn(temp_table['RA_d'].data, dtype=result_table_simbad['RA_d'].dtype,\
                                                       unit=result_table_simbad['RA_d'].unit,\
                                                       format=result_table_simbad['RA_d'].format,\
                                                       description=result_table_simbad['RA_d'].description)
                    temp_table['RA_d']  = replacement_col
                if temp_table['DEC_d'].unit != 'deg':
                    replacement_col     = MaskedColumn(temp_table['DEC_d'].data, dtype=result_table_simbad['DEC_d'].dtype,\
                                                       unit=result_table_simbad['DEC_d'].unit,\
                                                       format=result_table_simbad['DEC_d'].format,\
                                                       description=result_table_simbad['DEC_d'].description)
                    temp_table['DEC_d']  = replacement_col
                result_table_simbad = vstack([result_table_simbad, temp_table[0]])
            except:
                print(f'Error with element {(index)} of the sample')
                temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype,\
                                                                        result_table_simbad['DEC_d'].info.dtype))
                temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
                result_table_simbad = vstack([result_table_simbad, temp_table[0]])
                final_error += 1
            bar.update(index)

In [44]:
if query_simbad_flag:
    with progressbar.ProgressBar(min_value=limit_set, max_value=(limit_set*2 - 1)) as bar:
        for index, coord in enumerate(large_sample_coords[limit_set:(limit_set*2)]):
            temp_table      = Table()
            try:
                temp_table      = customSimbad.query_region(coord, radius=2.0*u.arcsec)
                if len(temp_table) == 0:
                    temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype,\
                                                                            result_table_simbad['DEC_d'].info.dtype))
                    temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
            except:
                temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype,\
                                                                        result_table_simbad['DEC_d'].info.dtype))
                temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
                #print(f'Error with element {(index + limit_set)} of the sample')
                query_error += 1
            try:
                if temp_table['RA_d'].unit != 'deg':
                    replacement_col     = MaskedColumn(temp_table['RA_d'].data, dtype=result_table_simbad['RA_d'].dtype,\
                                                       unit=result_table_simbad['RA_d'].unit,\
                                                       format=result_table_simbad['RA_d'].format,\
                                                       description=result_table_simbad['RA_d'].description)
                    temp_table['RA_d']  = replacement_col
                if temp_table['DEC_d'].unit != 'deg':
                    replacement_col     = MaskedColumn(temp_table['DEC_d'].data, dtype=result_table_simbad['DEC_d'].dtype,\
                                                       unit=result_table_simbad['DEC_d'].unit,\
                                                       format=result_table_simbad['DEC_d'].format,\
                                                       description=result_table_simbad['DEC_d'].description)
                    temp_table['DEC_d']  = replacement_col
                result_table_simbad = vstack([result_table_simbad, temp_table[0]])
            except:
                print(f'Error with element {(index + limit_set)} of the sample')
                temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype,\
                                                                        result_table_simbad['DEC_d'].info.dtype))
                temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
                result_table_simbad = vstack([result_table_simbad, temp_table[0]])
                final_error += 1
            bar.update(index + limit_set)

In [45]:
if query_simbad_flag:
    with progressbar.ProgressBar(min_value=(limit_set*2), max_value=(limit_set*3)) as bar:
        for index, coord in enumerate(large_sample_coords[(limit_set*2):(limit_set*3 + 1)]):
            temp_table      = Table()
            try:
                temp_table      = customSimbad.query_region(coord, radius=2.0*u.arcsec)
                if len(temp_table) == 0:
                    temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype,\
                                                                            result_table_simbad['DEC_d'].info.dtype))
                    temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
            except:
                temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype,\
                                                                        result_table_simbad['DEC_d'].info.dtype))
                temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
                # print(f'Error with element {(index + limit_set*2)} of the sample')
                query_error += 1
            try:
                if temp_table['RA_d'].unit != 'deg':
                    replacement_col     = MaskedColumn(temp_table['RA_d'].data, dtype=result_table_simbad['RA_d'].dtype,\
                                                       unit=result_table_simbad['RA_d'].unit,\
                                                       format=result_table_simbad['RA_d'].format,\
                                                       description=result_table_simbad['RA_d'].description)
                    temp_table['RA_d']  = replacement_col
                if temp_table['DEC_d'].unit != 'deg':
                    replacement_col     = MaskedColumn(temp_table['DEC_d'].data, dtype=result_table_simbad['DEC_d'].dtype,\
                                                       unit=result_table_simbad['DEC_d'].unit,\
                                                       format=result_table_simbad['DEC_d'].format,\
                                                       description=result_table_simbad['DEC_d'].description)
                    temp_table['DEC_d']  = replacement_col
                result_table_simbad = vstack([result_table_simbad, temp_table[0]])
            except:
                print(f'Error with element {(index + limit_set*2)} of the sample')
                temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype,\
                                                                        result_table_simbad['DEC_d'].info.dtype))
                temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
                result_table_simbad = vstack([result_table_simbad, temp_table[0]])
                final_error += 1
            bar.update(index + limit_set*2)

Query for `COSMOS` sources.

In [46]:
if query_simbad_flag:
    limit_set_cosmos = int(np.floor((np.shape(large_sample_z)[0] - np.shape(medium82_sample_z)[0])/3))

In [47]:
if query_simbad_flag:
    with progressbar.ProgressBar(min_value=(limit_set*3 + 1), max_value=(limit_set*3 + limit_set_cosmos)) as bar:
        for index, coord in enumerate(large_sample_coords[(limit_set*3 + 1):(limit_set*3 + 1 + limit_set_cosmos)]):
            temp_table      = Table()
            try:
                temp_table      = customSimbad.query_region(coord, radius=2.0*u.arcsec)
                if len(temp_table) == 0:
                    temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype,\
                                                                            result_table_simbad['DEC_d'].info.dtype))
                    temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
            except:
                # print(f'Error with element {(index + limit_set*3 + 1)} of the sample')
                temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype,\
                                                                        result_table_simbad['DEC_d'].info.dtype))
                temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
                query_error += 1
            try:
                if temp_table['RA_d'].unit != 'deg':
                    replacement_col     = MaskedColumn(temp_table['RA_d'].data, dtype=result_table_simbad['RA_d'].dtype,\
                                                       unit=result_table_simbad['RA_d'].unit,\
                                                       format=result_table_simbad['RA_d'].format,\
                                                       description=result_table_simbad['RA_d'].description)
                    temp_table['RA_d']  = replacement_col
                if temp_table['DEC_d'].unit != 'deg':
                    replacement_col     = MaskedColumn(temp_table['DEC_d'].data, dtype=result_table_simbad['DEC_d'].dtype,\
                                                       unit=result_table_simbad['DEC_d'].unit,\
                                                       format=result_table_simbad['DEC_d'].format,\
                                                       description=result_table_simbad['DEC_d'].description)
                    temp_table['DEC_d']  = replacement_col
                result_table_simbad = vstack([result_table_simbad, temp_table[0]])
            except:
                print(f'Error with element {(index + limit_set*3 + 1)} of the sample')
                temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype,\
                                                                        result_table_simbad['DEC_d'].info.dtype))
                temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
                result_table_simbad = vstack([result_table_simbad, temp_table[0]])
                final_error += 1
            bar.update(index + limit_set*3 + 1)

In [48]:
if query_simbad_flag:
    with progressbar.ProgressBar(min_value=(limit_set*3 + 1 + limit_set_cosmos), max_value=(limit_set*3 + limit_set_cosmos*2)) as bar:
        for index, coord in enumerate(large_sample_coords[(limit_set*3 + 1 + limit_set_cosmos):(limit_set*3 + 1 + limit_set_cosmos*2)]):
            temp_table      = Table()
            try:
                temp_table      = customSimbad.query_region(coord, radius=2.0*u.arcsec)
                if len(temp_table) == 0:
                    temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype,\
                                                                            result_table_simbad['DEC_d'].info.dtype))
                    temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
            except:
                temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype,\
                                                                        result_table_simbad['DEC_d'].info.dtype))
                temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
                # print(f'Error with element {(index + limit_set*3 + 1 + limit_set_cosmos)} of the sample')
                query_error += 1
            try:
                if temp_table['RA_d'].unit != 'deg':
                    replacement_col     = MaskedColumn(temp_table['RA_d'].data, dtype=result_table_simbad['RA_d'].dtype,\
                                                       unit=result_table_simbad['RA_d'].unit,\
                                                       format=result_table_simbad['RA_d'].format,\
                                                       description=result_table_simbad['RA_d'].description)
                    temp_table['RA_d']  = replacement_col
                if temp_table['DEC_d'].unit != 'deg':
                    replacement_col     = MaskedColumn(temp_table['DEC_d'].data, dtype=result_table_simbad['DEC_d'].dtype,\
                                                       unit=result_table_simbad['DEC_d'].unit,\
                                                       format=result_table_simbad['DEC_d'].format,\
                                                       description=result_table_simbad['DEC_d'].description)
                    temp_table['DEC_d']  = replacement_col
                result_table_simbad = vstack([result_table_simbad, temp_table[0]])
            except:
                print(f'Error with element {(index + limit_set*3 + 1 + limit_set_cosmos)} of the sample')
                temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype,\
                                                                        result_table_simbad['DEC_d'].info.dtype))
                temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
                result_table_simbad = vstack([result_table_simbad, temp_table[0]])
                final_error += 1
            bar.update(index + limit_set*3 + 1 + limit_set_cosmos)

In [49]:
if query_simbad_flag:
    with progressbar.ProgressBar(min_value=(limit_set*3 + 1 + limit_set_cosmos*2), max_value=np.shape(large_sample_coords)[0]) as bar:
        for index, coord in enumerate(large_sample_coords[(limit_set*3 + 1 + limit_set_cosmos*2):]):
            temp_table      = Table()
            try:
                temp_table      = customSimbad.query_region(coord, radius=2.0*u.arcsec)
                if len(temp_table) == 0:
                    temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype,\
                                                                            result_table_simbad['DEC_d'].info.dtype))
                    temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
            except:
                temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype,\
                                                                        result_table_simbad['DEC_d'].info.dtype))
                temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
                # print(f'Error with element {(index + limit_set*3 + 1 + limit_set_cosmos*2)} of the sample')
                query_error += 1
            try:
                if temp_table['RA_d'].unit != 'deg':
                    replacement_col     = MaskedColumn(temp_table['RA_d'].data, dtype=result_table_simbad['RA_d'].dtype,\
                                                       unit=result_table_simbad['RA_d'].unit,\
                                                       format=result_table_simbad['RA_d'].format,\
                                                       description=result_table_simbad['RA_d'].description)
                    temp_table['RA_d']  = replacement_col
                if temp_table['DEC_d'].unit != 'deg':
                    replacement_col     = MaskedColumn(temp_table['DEC_d'].data, dtype=result_table_simbad['DEC_d'].dtype,\
                                                       unit=result_table_simbad['DEC_d'].unit,\
                                                       format=result_table_simbad['DEC_d'].format,\
                                                       description=result_table_simbad['DEC_d'].description)
                    temp_table['DEC_d']  = replacement_col
                result_table_simbad = vstack([result_table_simbad, temp_table[0]])
            except:
                print(f'Error with element {(index + limit_set*3 + 1 + limit_set_cosmos*2)} of the sample')
                temp_table      = Table(names=('RA_d', 'DEC_d'), dtype=(result_table_simbad['RA_d'].info.dtype,\
                                                                        result_table_simbad['DEC_d'].info.dtype))
                temp_table.add_row((coord.ra.deg, coord.dec.deg))
                temp_table.meta['description'] = 'Simbad_q'
                result_table_simbad = vstack([result_table_simbad, temp_table[0]])
                final_error += 1
            bar.update(index + limit_set*3 + 1 + limit_set_cosmos*2)

Query remaining sources

We save the query to a file for future executions.

In [50]:
if query_simbad_flag:
    result_table_simbad.info

In [51]:
if query_simbad_flag:
    result_table_simbad.write(cat_path + 'large_cat_simbad_query_ago2020.csv', format='ascii.csv',\
                              overwrite=True, serialize_method='data_mask')

Steps to create a copy of table to save it as `fits` file.

In [52]:
if query_simbad_flag:
    copy_simbad_to_write = result_table_simbad
    str_id = copy_simbad_to_write['MAIN_ID'].astype('str')
    copy_simbad_to_write.replace_column('MAIN_ID', str_id)

In [53]:
if query_simbad_flag:
    copy_simbad_to_write.info

In [54]:
if query_simbad_flag:
    copy_simbad_to_write.write(cat_path + 'large_cat_simbad_query_ago2020.fits', format='fits',\
                               overwrite=True, serialize_method='data_mask')

To save running time, we can load the data from a file.

In [55]:
#if load_simbad_flag:
#    result_table_simbad = Table.read(cat_path + 'large_cat_simbad_query.csv', format='ascii.csv')

In [56]:
if load_simbad_flag:
    result_table_simbad     = Table.read(cat_path + 'large_cat_simbad_query_ago2020.fits');

In [57]:
# result_table_simbad.info

From this point, we merge the data from the query to `simbad` with the  
values from this notebook (**Inayoshi et al., 2020** and **SDSS+FIRST**).  
In order to do this, we convert the data into `astropy` columns, and then  
into `astropy` tables. They will be ready to be exported.

In [58]:
if create_simbad_inayoshi_flag:
    coords_simbad_inayoshi = SkyCoord(result_table_simbad['RA_d'], result_table_simbad['DEC_d'], unit=u.deg)

In [59]:
if create_simbad_inayoshi_flag:
    np.shape(result_table_simbad)

In [60]:
if create_simbad_inayoshi_flag:
    result_table_simbad.add_columns([column_cat_index, column_cat_name, column_cat_ra, column_cat_dec, column_z_own,\
                                     column_z_own_err, column_L_14GHz_up, column_f_20cm, column_f_20cm_err, column_f_6cm,\
                                     column_f_6cm_err, column_f_3GHz, column_f_3GHz_err, column_f_15GHz, column_f_15GHz_err,\
                                     column_f_xmm, column_f_xmm_err, column_f_250GHz, column_f_250GHz_err, column_mass_1450,\
                                     column_flx_lim, column_origin, column_refs])

In [61]:
if create_simbad_inayoshi_flag:
    str_id = result_table_simbad['MAIN_ID'].astype('str')
    result_table_simbad.replace_column('MAIN_ID', str_id)

In [62]:
# copy_table = result_table_simbad.filled(fill_value=np.nan)

We write the table into a file. It can be `.fits`, `.votable`, etc.

In [63]:
#copy_table.write('high_z_qsos.ecsv', format='ascii.ecsv', overwrite=True, serialize_method='data_mask')

In [64]:
if create_simbad_inayoshi_flag:
    result_table_simbad.write(cat_path + 'large_cat_simbad_query_inayoshi_ago2020.fits', format='fits',\
                              overwrite=True, serialize_method='data_mask')

In [65]:
if create_simbad_inayoshi_flag:
    result_table_simbad.write(cat_path + 'large_cat_simbad_query_inayoshi_ago2020.csv', format='ascii.csv',\
                              overwrite=True, serialize_method='data_mask')

To save running time, we can load the data from a file.

In [66]:
#if read_simbad_inayoshi_flag:
#    result_table_simbad = Table.read(cat_path + 'large_cat_simbad_query_inayoshi.csv', format='ascii.csv')

In [67]:
if read_simbad_inayoshi_flag:
    result_table_simbad = Table.read(cat_path + 'large_cat_simbad_query_inayoshi_ago2020.fits');

In [68]:
if read_simbad_inayoshi_flag or create_simbad_inayoshi_flag:
    result_table_simbad.info

Query the objects of the table in other catalogs and services.

In [69]:
#from astroquery.heasarc import Heasarc
#Heasarc.query_mission_cols(mission='radio')
#tabb = Heasarc.query(large_sample_names, mission='radio', timeout=90)

In [70]:
customNed        = Ned()
fields_to_remove = ['No.', 'Photometry Measurement', 'Uncertainty', 'Units', 'Significance',\
                    'Published frequency', 'Frequency Mode', 'Coordinates Targeted', 'Spatial Mode', 'Qualifiers', 'Comments']

Querying sources with name in `Ned`.

In [71]:
# empty_counter = 0
# res_tab       = {}
# for name in large_sample_names:
#     try:
#         res_tab[name] = customNed.get_table(name, output_table_format=1)
#         res_tab[name].remove_columns(fields_to_remove)
#     except:
#         res_tab[name] = Table()
#         empty_counter += 1

As before, we can do it with coordinates.  

First, we query the coordinates. If we found something,  
we use the name of the source to obtain it photometry.

In [72]:
if read_simbad_inayoshi_flag or create_simbad_inayoshi_flag:
    coords_simbad_inayoshi = SkyCoord(result_table_simbad['RA_d'], result_table_simbad['DEC_d'], unit=u.deg)

In [73]:
empty_tab_name_counter  = 0
empty_tab_photo_counter = 0
error_tab_name_counter  = 0
error_tab_photo_counter = 0
ned_tables              = {}
ned_info                = {}
ned_names               = []
#ned_names               = np.array([''  for x in np.arange(np.shape(large_sample_names)[0])])

In [74]:
if query_ned_names_flag:
    with progressbar.ProgressBar(min_value=0, max_value=np.shape(large_sample_coords)[0]) as bar:
        for index, coord in enumerate(large_sample_coords):
            try:
                init_table            = customNed.query_region(large_sample_coords[index], radius=2.0*u.arcsec)
                if len(init_table) == 0:
                    # init_table        = Table(names=('Object Name', 'RA', 'DEC'), dtype=('str', 'float', 'float'), masked=True)
                    # init_table.add_row(('No Name', large_sample_coords[index].ra.deg,\
                    # large_sample_coords[index].dec.deg), mask=[True, False, False])
                    first_row         = [['No Name'], [large_sample_coords[index].ra.deg], [large_sample_coords[index].dec.deg]]
                    init_table        = Table(first_row, names=('Object Name', 'RA', 'DEC'), dtype=('str', 'float', 'float'), masked=True)
                    ned_info[index]   = init_table
                    ned_names.append('No Name')
                    empty_tab_name_counter += 1
                    bar.update(index)
                    continue
                # init_table.remove_columns(['Magnitude and Filter', 'Positions', 'Diameter Points'])
                used_source_idx   = np.nanargmin(init_table['Separation'])  # Index of element with lowest separation from coords
                ned_info[index]   = Table(init_table[used_source_idx])
                init_name         = init_table['Object Name'][used_source_idx]
                # ned_names[index]  = init_name
                ned_names.append(init_name)
            except:
                # init_table        = Table(names=('Object Name', 'RA', 'DEC'), dtype=('str', 'float', 'float'), masked=True)
                # init_table.add_row(('No Name', large_sample_coords[index].ra.deg,\
                # large_sample_coords[index].dec.deg), mask=[True, False, False])
                first_row         = [['No Name'], [large_sample_coords[index].ra.deg], [large_sample_coords[index].dec.deg]]
                init_table        = Table(first_row, names=('Object Name', 'RA', 'DEC'), dtype=('str', 'float', 'float'), masked=True)
                ned_info[index]   = init_table
                ned_names.append('No Name')
                error_tab_name_counter += 1
            bar.update(index)

In [75]:
# init_table            = customNed.query_region(coords_simbad_inayoshi[18022], radius=4.0*u.arcsec)

In [76]:
if query_ned_names_flag:
    ned_names = np.array(ned_names)

In [77]:
if query_ned_names_flag:
    ned_redshifts     = []
    ned_redshifts_unc = []
    with progressbar.ProgressBar(min_value=0, max_value=np.shape(ned_names)[0]) as bar:
        for key in ned_info:
            if np.shape(ned_info[key].colnames)[0] < 4:
                ned_redshifts.append(np.nan)
                ned_redshifts_unc.append(np.nan)
                bar.update(key)
                continue
            ned_redshifts.append(ned_info[key]['Redshift'])
            ned_name_temp = ned_info[key]['Object Name'][0]
            if ned_name_temp == 'No Name':
                ned_redshifts.append(np.nan)
                ned_redshifts_unc.append(np.nan)
                bar.update(key)
                continue
            try:
                temp_z_table = customNed.get_table(ned_name_temp, table='redshifts')
                ned_redshifts_unc.append(temp_z_table['Published Redshift Uncertainty'][0])
            except:
                ned_redshifts_unc.append(np.nan)
            bar.update(key)
    ned_redshifts     = np.array(ned_redshifts)
    ned_redshifts_unc = np.array(ned_redshifts_unc)

In [78]:
# np.sum(np.isfinite(ned_redshifts_unc))

In [79]:
# np.sum(ned_redshifts_unc >= 0)

In [80]:
counter_count = 0
indices_non   = []
for index, name in enumerate(ned_names):
    if name != 'No Name':
        counter_count += 1
for index, name in enumerate(ned_names):
    if name == 'No Name':
        indices_non.append(index)

In [81]:
if query_ned_names_flag:
    ned_names[1234]

In [82]:
# name_tab_temp   = customNed.get_table(str(ned_names[1]))

In [83]:
if query_ned_photometry_flag:
    with progressbar.ProgressBar(min_value=0, max_value=np.shape(ned_names)[0]) as bar:
        for index, name in enumerate(ned_names):
            try:
                if name == 'No Name':
                    # phot_table        = Table(names=('Observed Passband',), dtype=('str',), masked=True)
                    # phot_table.add_row(('No Passband',), mask=(True,))
                    first_row         = [[0], ['No Passband']]
                    phot_table        = Table(first_row, names=('No.', 'Observed Passband'), dtype=('int', 'str'), masked=True)
                    ned_tables[index] = phot_table
                    empty_tab_photo_counter += 1
                    bar.update(index)
                    continue
                phot_table            = customNed.get_table(name, table='photometry', output_table_format=3)
                # phot_table.remove_columns(fields_to_remove)
                ned_tables[index]     = phot_table
            except:
                # phot_table            = Table(names=('Observed Passband',), dtype=('str',), masked=True)
                # phot_table.add_row(('No Passband',), mask=(True,))
                first_row             = [[0], ['No Passband']]
                phot_table            = Table(first_row, names=('No.', 'Observed Passband'), dtype=('int', 'str'), masked=True)
                ned_tables[index]     = phot_table
                error_tab_photo_counter += 1
            bar.update(index)

In [84]:
# name_tab_temp   = customNed.get_table('Himiko', table='photometry')

Save multiple data tables into one file with `pandas`

In [85]:
# ned_names[4]

In [86]:
if create_simbad_inayoshi_ned_flag:
    pass_band_name = ned_tables[key]['Observed Passband'].astype('str')
    ned_tables[key].replace_column('Observed Passband', pass_band_name)

We gather the names and frequencies of all columns present in tables from `Ned`.

In [87]:
if create_phot_bands_list_flag:
    band_names_array = []
    
    n_t_str          = '\t'
    
    with progressbar.ProgressBar(min_value=0, max_value=np.shape(ned_names)[0]) as bar:
        for key in ned_tables:
            if ned_tables[key]['Observed Passband'][0] == 'No Passband':
                bar.update(key)
                continue
            pass_band_name = ned_tables[key]['Observed Passband'].astype('str')
            ned_tables[key].replace_column('Observed Passband', pass_band_name)
            temp_table    = ned_tables[key]['Observed Passband', 'Frequency'].as_array().data
            for temp_pair in temp_table:
                band_names_array.append([str(temp_pair[0]), f'{temp_pair[1]:.4e}'])
            bar.update(key)

Unique rows (name + frequency pair) are retrieved from the previous list.

In [88]:
useful_number = 0
for key in ned_tables:
    if np.shape(ned_tables[key].colnames)[0] >= 3:
        useful_number += 1
print(useful_number)

0


In [89]:
if create_phot_bands_list_flag:
    unique_rows_band_names_array = np.unique(band_names_array, axis=0)
    # Order by frequency
    unique_rows_band_names_array = unique_rows_band_names_array[unique_rows_band_names_array[:, 1].astype(np.float).argsort()]

We obtain the size of this new array. It represents the number of  
unique passband configurations gathered from querying `Ned`.

In [90]:
if create_phot_bands_list_flag:
    print(np.shape(unique_rows_band_names_array))

In [91]:
# unique_rows_band_names_array

Saving this list into a plain text file.

In [92]:
if create_phot_bands_list_flag:
    # np.savetxt(cat_path + 'all_ned_band_names.txt', unique_rows_band_names_array, fmt='%s %s', delimiter='\t')
    # np.savetxt(cat_path + 'all_ned_band_names_2_5arcsec.txt', unique_rows_band_names_array, fmt='%s %s', delimiter='\t')
    np.savetxt(cat_path + 'all_ned_band_names_ago2020.txt', unique_rows_band_names_array, fmt='%s %s', delimiter='\t')

This new file can also be loaded.

In [93]:
if load_phot_bands_list_flag:
    unique_rows_band_names_array = np.genfromtxt(cat_path + 'all_ned_band_names_ago2020.txt', delimiter='\t', dtype=str)

Now, we create a unique table from all the individual photometry tables obtained after  
querying `Ned`. We discard measurements which have already been reported (for each individual source).

We leave uncertainty values as string to retain information about possible upper/lower limits.

In [94]:
if order_ned_photometry_flag:
    limit_set_ned = int(np.floor(np.shape(ned_names)[0]/4))

In [95]:
# order_ned_photometry_flag = True

In [96]:
if order_ned_photometry_flag:
    temp_table_ned_photo             = Table()
    chunk_size                       = 300  # Number of elements to calculate before dumping results to external table
    with progressbar.ProgressBar(min_value=0, max_value=np.shape(ned_names)[0]) as bar:
        for index, source_name in enumerate(ned_names):  # Some names will be 'No Name'
            band_names_str           = []
            column_names_str         = []
            band_frequencies         = []
            measure_names            = ned_tables[index].colnames[1:]
            # init_table = Table(names=('MAIN_ID',), dtype=('str',), masked=True)
            cord_str                 = large_sample_coords[index].to_string('decimal')
            init_table               = Table(data=np.array([index, cord_str, source_name]),\
                                             names=('INDEX', 'COORD', 'MAIN_ID'),\
                                             dtype=('int', 'str', 'str'), masked=True)
            if source_name == 'No Name':
                if index == 0 or (index % chunk_size == 1 and index > 1):
                    init_table_large     = Table(init_table)
                    bar.update(index)
                    continue
                init_table_large     = vstack([init_table_large, init_table])
                if index % chunk_size == 0 and index > 0:
                    temp_table_ned_photo = vstack([temp_table_ned_photo, init_table_large])
                bar.update(index)
                continue
            if len(measure_names) == 0 or len(measure_names) == 1:
                # init_table = Table(('No Name',), names=('MAIN_ID',), dtype=('str',), masked=True)
                # init_table.add_row((source_name,), mask=(True,))
                if index == 0 or (index % chunk_size == 1 and index > 1):
                    init_table_large     = Table(init_table)
                    bar.update(index)
                    continue
                init_table_large     = vstack([init_table_large, init_table])
                if index % chunk_size == 0 and index > 0:
                    temp_table_ned_photo = vstack([temp_table_ned_photo, init_table_large])
                    # result_table_simbad_copy = join(result_table_simbad_copy, init_table_large, keys='COORD', join_type='outer')
                bar.update(index)
                continue
            # init_table = Table((str(source_name),), names=('MAIN_ID',), dtype=('str',), masked=True)
            # init_table.add_row((str(source_name),))
            for row in ned_tables[index]:
                # band_name_str        = re.sub(r' \(.*', '', str(row['Observed Passband'].decode('utf-8')))  # Eliminate differences
                # band_name_str        = str(row['Observed Passband'].decode('utf-8'))  # Eliminate differences
                if type(row['Observed Passband']) != bytes:
                    band_name_str        = str(row['Observed Passband'])  # Eliminate differences
                if type(row['Observed Passband']) == bytes:
                    band_name_str        = str(row['Observed Passband'].decode('utf-8'))  # Eliminate differences
                # if str(band_name_str) not in band_names_str and row['Frequency'] not in band_frequencies:
                if str(band_name_str) not in band_names_str:
                    band_frequencies.append(row['Frequency'])
                    band_names_str.append(str(band_name_str))
                    column_name_flux = 'Flux Density ' + band_name_str
                    column_name_err  = 'NED Uncertainty ' + band_name_str
                    if column_name_flux not in column_names_str:
                        column_names_str.append(column_name_flux)
                        column_flux  = MaskedColumn(row['Flux Density'], name=column_name_flux,\
                                                    unit=ned_tables[index]['Flux Density'].unit, dtype='float')
                        column_err   = MaskedColumn(row['NED Uncertainty'], name=column_name_err, dtype='str')
                        init_table.add_columns((column_flux, column_err))
            #init_table.remove_column('MAIN_ID')
            if index == 0 or (index % chunk_size == 1 and index > 1):
                init_table_large     = Table(init_table)
                bar.update(index)
                continue
            init_table_large         = vstack([init_table_large, init_table])
            if index % chunk_size == 0 and index > 0:
                temp_table_ned_photo = vstack([temp_table_ned_photo, init_table_large])
                # result_table_simbad_copy = join(result_table_simbad_copy, init_table_large, keys='COORD', join_type='outer')
            bar.update(index)
    temp_table_ned_photo = vstack([temp_table_ned_photo, init_table_large])

In [97]:
if order_ned_photometry_flag:
    len(measure_names)

This new merged column shows measurements for the following number of bands.

In [98]:
if order_ned_photometry_flag:
    len(temp_table_ned_photo.colnames)

For data completeness, we mask entries for `MAIN_ID` and `COORD` columns.

In [99]:
if create_simbad_inayoshi_ned_flag:
    column_coord = MaskedColumn(temp_table_ned_photo['COORD'], name='COORD',\
                                mask=np.array(temp_table_ned_photo['COORD'] == ''))
    column_id    = MaskedColumn(temp_table_ned_photo['MAIN_ID'], name='MAIN_ID',\
                                description='id_obj', mask=np.array(temp_table_ned_photo['MAIN_ID'] == 'No Name'))

In [100]:
if create_simbad_inayoshi_ned_flag:
    temp_table_ned_photo.replace_column('COORD', column_coord)
    temp_table_ned_photo.replace_column('MAIN_ID', column_id)

We add a column with the redshift uncertainties from Ned.

In [101]:
if create_simbad_inayoshi_ned_flag:
    temp_table_ned_photo.add_column(MaskedColumn(ned_redshifts_unc, name='ned_z_err', mask=np.array(ned_redshifts_unc <= 0)))

We transform `astropy` tables into `pandas` data frames for further calculations.

In [102]:
if create_simbad_inayoshi_ned_flag:
    df_ned    = temp_table_ned_photo.to_pandas()

In [103]:
if create_simbad_inayoshi_ned_flag:
    df_ned.to_hdf(cat_path + 'large_cat_inayoshi_ned_ago2020.h5', 'df')

In [104]:
if create_simbad_inayoshi_ned_flag:
    df_simbad = result_table_simbad.to_pandas()

In this point, we can merge both `simbad` and `Ned` tables into a larger one.

In [105]:
if create_simbad_inayoshi_ned_flag:
    merged_ned_simbad = pd.merge(df_simbad, df_ned, on='INDEX')

In [106]:
# merged_ned_simbad.columns[0:40]

Remove duplicates in name.

In [107]:
if create_simbad_inayoshi_ned_flag:
    removed_dup = pd.concat([ 
        merged_ned_simbad[merged_ned_simbad['MAIN_ID_x'].isna()], 
        merged_ned_simbad[~merged_ned_simbad['MAIN_ID_x'].isna()].drop_duplicates(subset='MAIN_ID_x', keep='first') 
    ]).reset_index(drop=True, inplace=False)

In [108]:
if create_simbad_inayoshi_ned_flag:
    print(np.shape(removed_dup))

We save this new table into a `HDF5`-format file for future access.

In [109]:
if create_simbad_inayoshi_ned_flag:
    removed_dup.to_hdf(cat_path + 'large_cat_simbad_inayoshi_ned_ago2020.h5', 'df')
    # merged_ned_simbad.to_hdf(cat_path + 'large_cat_simbad_inayoshi_ned_all_cols.h5', 'df')

If needed, we can read the table from an external file to avoid extra running time.

In [110]:
if read_simbad_inayoshi_ned_flag:
    power_test = pd.read_hdf(cat_path + 'large_cat_simbad_inayoshi_ned_ago2020.h5', 'df')

And this `pandas` data frame can be transformed into an `astropy` table.

In [111]:
# power_table = Table.from_pandas(power_test)